In [1]:
!pip install bidict

In [1]:
import pickle
import os, glob

# more than n days
import json, pickle
from bidict import bidict  # bidict = bi-directional dictionary. "key <-> value" .inverse['value'] to use value as key.
from onoff import OnOff
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
from collections import defaultdict, Counter
from datetime import datetime
import statistics
import csv

In [3]:
dir_pickle = "/nas001/users/hyunchan.moon/app_filter/"

In [4]:
if not os.path.exists('data'):
    os.mkdir('data')
if not os.path.exists('stats'):
    os.mkdir('stats')

In [5]:
from onoff import OnOff
pickle_f = glob.glob(os.path.join(dir_pickle,"*.pickle"))[0]

with open(pickle_f, "rb") as inFile:
    pp_onoffs = pickle.load(inFile)

In [6]:
merged_processed_OnOffs_per_user = {}
for tester_email, processed_OnOffs_per_day in pp_onoffs.items():
    merged_processed_OnOffs = []
    for date, OnOff_list in processed_OnOffs_per_day.items():
        merged_processed_OnOffs.extend(OnOff_list)
    merged_processed_OnOffs_per_user[tester_email] = merged_processed_OnOffs
    
    
with open("data/testers.json") as file:
    TESTERS = json.load(file)

with open("stats/app_category_dictionary.json","r") as file:
    CATEGORIES = json.load(file)

unique_categories = list(set(value for value in CATEGORIES.values() if value))
unique_categories.sort()


In [7]:
package_name_dic = {'NOTIFICATION':'NT', 'typeWindowStateChanged':'WC', 'typeViewFocused':'VF', 'typeViewTextChanged':'TC', 'EXTERNAL_SENSOR':'EXTERNAL_SENSOR'}

In [13]:
extract_pickle_list = {'pacakages': {"TC":0.3, 'WC':0.6}, 'extract_pack': dict()}

In [26]:
def extract_category_top_ratio(extract_list:dict()):
    extract_summary_top_ratio = dict()
    

    for tester_email, OnOffs in merged_processed_OnOffs_per_user.items():
        category_counters_day = {
            "NT": defaultdict(list),
            "WC": defaultdict(list),
            "VF": defaultdict(list),
            "TC": defaultdict(list),
            "All": defaultdict(list)
        }

        for single_OnOff in OnOffs:
            for i in range(len(single_OnOff.items)):
                single_OnOff_events_dict = json.loads(single_OnOff.events[i])
                timestamp = datetime.fromtimestamp(single_OnOff_events_dict['timestamp'] / 1000).date()
                if single_OnOff_events_dict['datumType'] == 'NOTIFICATION':
                    category_counters_day["NT"][timestamp].append(single_OnOff_events_dict['packageName'])
                elif single_OnOff_events_dict['deviceType'] == 'typeWindowStateChanged':
                    category_counters_day["WC"][timestamp].append(single_OnOff_events_dict['packageName'])
                elif single_OnOff_events_dict['deviceType'] == 'typeViewFocused':
                    category_counters_day["VF"][timestamp].append(single_OnOff_events_dict['packageName'])
                elif single_OnOff_events_dict['deviceType'] == 'typeViewTextChanged':
                    category_counters_day["TC"][timestamp].append(single_OnOff_events_dict['packageName'])
                category_counters_day["All"][timestamp].append(single_OnOff_events_dict['packageName'])

        # Calculate the average and standard deviation for each event within a category

        event_stats = {}

        days = len(category_counters_day["All"])

        for event_category, timestamps in category_counters_day.items():
            if event_category not in  extract_list.keys():
                continue
            event_stats[event_category] = {}
            for timestamp, events in timestamps.items():
                # make Counter for each day
                event_counts = Counter(events)
                for event, count in event_counts.items():
                    event_stats[event_category][event] = event_stats.get(event_category, {}).get(event, [])
                    event_stats[event_category][event].append(count)

        top_package_names = []

        for event_category, event_data in event_stats.items():
            if event_category not in extract_list.keys():
                continue
            n = int(days * extract_list[event_category])
            for event, counts in event_data.items():
                if len(counts) >= n:
                    top_package_names.append(event)

        not_in_dict = set()
        current = []
        for package_name in top_package_names:
            current.append(package_name)
            
        extract_summary_top_ratio[tester_email] = current
        
    return extract_summary_top_ratio


In [27]:
for k, v in  extract_pickle_list['pacakages'].items():
    extract_pickle_list['extract_pack'][k] = extract_category_top_ratio(extract_list={k:v})
    print({k:v})

{'TC': 0.3}
{'WC': 0.6}


In [31]:
extract_pickle_list

{'pacakages': {'TC': 0.3, 'WC': 0.6},
 'extract_pack': {'TC': {'smartspeakertester1@gmail.com': ['com.kakao.talk',
    'com.samsung.android.app.notes',
    'com.samsung.android.calendar',
    'com.instagram.android',
    'com.nhn.android.nmap',
    'kr.co.quicket',
    'com.samsung.android.messaging'],
   'smartspeakertester2@gmail.com': ['com.google.android.youtube',
    'com.android.chrome',
    'com.google.android.apps.youtube.music',
    'com.google.android.calendar',
    'droom.sleepIfUCan',
    'com.Slack',
    'net.daum.android.map',
    'com.kakao.talk',
    'com.coupang.mobile',
    'notion.id'],
   'smartspeakertester3@gmail.com': ['com.kakao.talk',
    'mars.nomad.com.blinddate2018',
    'com.samsung.android.app.galaxyfinder',
    'com.hsv.freeadblockerbrowser',
    'com.samsung.android.dialer',
    'com.sec.android.app.sbrowser',
    'com.samsung.android.messaging',
    'com.nhn.android.nmap',
    'com.nhn.android.search',
    'com.coupang.mobile',
    'com.samsung.android.

In [29]:
processed_OnOffs_per_day_per_user_new = {}

In [ ]:
for tester_email, processed_OnOffs_per_day in pp_onoffs.items():
    
    processed_OnOffs_per_day_new = defaultdict(list)

    for date, OnOff_list in processed_OnOffs_per_day.items():
        for single_OnOff in OnOff_list:
            processed_single_OnOff_items_new = []
            processed_single_OnOff_events_new = []
            category_counters_day = {
            "NT": [],
            "WC": [],
            "VF": [],
            "TC": []
            }
            for i in range(len(single_OnOff.items)):
                single_OnOff_events_dict = json.loads(single_OnOff.events[i])
                if package_name_dic[single_OnOff_events_dict['datumType']] in list(extract_pickle_list['pacakages'].keys()):
                    category_counters_day[package_name_dic[single_OnOff_events_dict['datumType']]].append(i)
                if 'deviceType' not in  single_OnOff_events_dict.keys():
                    continue
                else:
                    if package_name_dic[single_OnOff_events_dict['deviceType']] in list(extract_pickle_list['pacakages'].keys()):
                        category_counters_day[package_name_dic[single_OnOff_events_dict['deviceType']]].append(i)
                    else:
                        continue

            for vp in list(extract_pickle_list['pacakages'].keys()):
                if len(extract_pickle_list['extract_pack'][vp][tester_email]) == 0:
                    continue
   
                if len(category_counters_day[vp]) ==0:
                    continue
                else:
                    for vi in category_counters_day[vp]:
                        valid_events = json.loads(single_OnOff.events[vi])
                        if valid_events['packageName'] in extract_pickle_list['extract_pack'][vp][tester_email]:
                            processed_single_OnOff_items_new.append(single_OnOff.items[vi])
                            processed_single_OnOff_events_new.append(single_OnOff.events[vi])

            if processed_single_OnOff_items_new:  # 빈 On-Off sequence들은 제거합니다.
                processed_OnOffs_per_day_new[date].append(
                    OnOff(
                        single_OnOff.ontime,
                        single_OnOff.offtime,
                        single_OnOff.userpresent,
                        tuple(processed_single_OnOff_items_new),
                        tuple(processed_single_OnOff_events_new),
                    )
                )
        processed_OnOffs_per_day_per_user_new[tester_email] = processed_OnOffs_per_day_new


In [94]:
with open("data/processed_OnOffs_per_day_per_user_WC_TC_top5_.pickle", "wb") as outFile:
        pickle.dump(processed_OnOffs_per_day_per_user_new, outFile)